# Ditto Training Pipeline

This notebook sets up the environment and runs the training pipeline for Ditto on Google Colab Pro.

## 1. Mount Google Drive
Mount your Google Drive to save checkpoints and access the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Setup Environment
Install necessary dependencies.

In [ ]:
!pip install -r requirements.txt
!pip install accelerate

## 3. Configuration
Configure the training parameters. The default model is `Wan-AI/Wan2.1-VACE-1.3B`.

In [ ]:
#@title Training Configuration

dataset_base_path = "/content/drive/MyDrive/Ditto-1M/videos" #@param {type:"string"}
dataset_metadata_path = "/content/drive/MyDrive/csvs_for_DiffSynth/xxx.csv" #@param {type:"string"}
output_path = "/content/drive/MyDrive/exps/ditto" #@param {type:"string"}
model_id = "Wan-AI/Wan2.1-VACE-1.3B" #@param {type:"string"}
num_epochs = 5 #@param {type:"integer"}
learning_rate = "1e-4" #@param {type:"string"}

print(f"Configuration:")
print(f"  Dataset Base Path: {dataset_base_path}")
print(f"  Metadata Path: {dataset_metadata_path}")
print(f"  Output Path: {output_path}")
print(f"  Model ID: {model_id}")
print(f"  Epochs: {num_epochs}")
print(f"  Learning Rate: {learning_rate}")

## 4. Run Training
Execute the training script with the configured parameters.

In [ ]:
!chmod +x train.sh
!./train.sh \
  --dataset_base_path "{dataset_base_path}" \
  --dataset_metadata_path "{dataset_metadata_path}" \
  --output_path "{output_path}" \
  --model_id "{model_id}" \
  --num_epochs "{num_epochs}" \
  --learning_rate "{learning_rate}"